In [1]:
from keras.models import load_model
model = load_model('output.h5')
model.summary()

Using TensorFlow backend.
C:\Users\SAI BALAJI OFFICE\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 544, 8)            80000     
_________________________________________________________________
gru_1 (GRU)                  (None, 544, 16)           1200      
_________________________________________________________________
gru_2 (GRU)                  (None, 544, 8)            600       
_________________________________________________________________
gru_3 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
Total params: 81,961
Trainable params: 81,961
Non-trainable params: 0
_________________________________________________________________


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from scipy.spatial.distance import cdist
# from tf.keras.models import Sequential  # This does not work!
from keras.models import Sequential
from keras.layers import Dense, GRU, Embedding
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import download
import imdb
imdb.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


In [3]:
x_train_text, y_train = imdb.load_data(train=True) #loading train data
x_test_text, y_test = imdb.load_data(train=False) # loading test data
print("Train-set size: ", len(x_train_text))
print("Test-set size:  ", len(x_test_text))

Train-set size:  25000
Test-set size:   25000


In [8]:
data_text = x_train_text + x_test_text
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(data_text)
tokenizer.word_index

x_train_tokens = tokenizer.texts_to_sequences(x_train_text)
x_test_tokens = tokenizer.texts_to_sequences(x_test_text) # converting text data into tokens
#print(x_train_text[2])
#print(np.array(x_train_tokens[0]))
num_tokens = np.array([len(tokens) for tokens in x_train_tokens + x_test_tokens])
print(np.mean(num_tokens))
print(np.max(num_tokens))
print(np.min(num_tokens))

221.27714
2209
6


In [9]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
print(max_tokens)
print(str(np.sum(num_tokens < max_tokens) / len(num_tokens) * 100) +' %' )

544
94.53 %


In [10]:
pad = 'pre'
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,
                            padding=pad, truncating=pad)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)
num_tokens_pad = np.array([len(tokens) for tokens in x_train_pad + x_test_pad])
result = model.evaluate(x_test_pad, y_test)

25000/25000 [==============================] - 220s 9ms/step


In [20]:
positive_review='''its very good movie'''

negative_review='''its very bad movie'''

text=[positive_review,negative_review]
tokens = tokenizer.texts_to_sequences(text) # we need to tokenize

tokens_pad = pad_sequences(tokens, maxlen=max_tokens,
                           padding=pad, truncating='pre')
# padding
a=model.predict(tokens_pad)[0]
b=model.predict(tokens_pad)[1]
print(a,b)

if a > 0.60: # I am thresholding it at 60%
    print('Positive Review with a score of {} %'.format(a[0]*100))
else:
    print('Negative Review ')


if b < 0.50: # I am thresholding it at 50%
    print('Positive Review ')
else:
    print('Negative Review with a score of {} %'.format(b[0]*100))




[0.86208284] [0.59047353]
Positive Review with a score of 86.2082839012146 %
Negative Review with a score of 59.04735326766968 %
